Try a Support Vector Machine regressor (sklearn.svm.SVR), with various hyperparameters such as kernel="linear" (with various values for the C hyperparameter) or kernel="rbf" (with various values for the C and gamma hyperparameters). Don’t worry about what these hyperparameters mean for now.

How does the best SVR predictor perform?


My dataset set has alredy been splitted for me into train set and test set

In [1]:
# Libary
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from  sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from sklearn.inspection import permutation_importance
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

In [2]:
#Import my dataset
train_set = pd.read_csv("C:/Users/HP/Desktop/ML with scikit-learn, keras and tensorflow/Data/Titanic/train.csv")
test_set = pd.read_csv("C:/Users/HP/Desktop/ML with scikit-learn, keras and tensorflow/Data/Titanic/test.csv")
label_set = pd.read_csv("C:/Users/HP/Desktop/ML with scikit-learn, keras and tensorflow/Data/Titanic/gender_submission.csv")
# train_set.head(7)
test_set.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# Clean my Train set

In [3]:
# Check for NaN
train_set.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
# Columns to drop
drop_columns = ['PassengerId', 'Cabin', 'Name', 'Ticket']

# Drop column
drop_set = train_set.drop(drop_columns, axis=1)
drop_set['Embarked'].value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [51]:
# List categorical variable and numerical varible
cat_set = ['Sex', 'Embarked']
num_set = ['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

# Set my numerical dataset
drop_set_num = drop_set[num_set]
drop_set_num_attribs = list(drop_set_num)
drop_set_num_attribs

# Set my categorical variable
drop_set_cat =drop_set[cat_set]
drop_set_cat_attribs = list(drop_set_cat)
#Create a list of cat dataframe
# drop_set_cat_attribs = list(drop_set_cat_trans)
# drop_set_cat_attribs

# Clean my cat Nan rows and convert to dataframe
pipeline = Pipeline([
    ("imputer",SimpleImputer(strategy="most_frequent")),
    ('ord', OrdinalEncoder())
])
# imputer = SimpleImputer(strategy="most_frequent")
drop_set_cat_trans = pipeline.fit_transform(drop_set_cat)
drop_set_cat_trans = pd.DataFrame(drop_set_cat_trans, columns=['Sex', 'Embarked'])


# # Deal with NaN in Age column
full_pipeline = ColumnTransformer([
    ('num', SimpleImputer(strategy='median'), drop_set_num_attribs),
    ('ord', pipeline, drop_set_cat_attribs),
])


# Transform my dataset
train_set_prepared = full_pipeline.fit_transform(drop_set)
# Create a column name for my prepared datset
train_set_prepared = pd.DataFrame(train_set_prepared, columns= ['Survived', 'Pclass', 'Age', 'SibSp','Parch', 'Fare', 'Sex','Embarked'])

train_set_prepared.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
0,0.0,3.0,22.0,1.0,0.0,7.2500,1.0,2.0
1,1.0,1.0,38.0,1.0,0.0,71.2833,0.0,0.0
2,1.0,3.0,26.0,0.0,0.0,7.9250,0.0,2.0
3,1.0,1.0,35.0,1.0,0.0,53.1000,0.0,2.0
4,0.0,3.0,35.0,0.0,0.0,8.0500,1.0,2.0


In [60]:
y = train_set_prepared['Survived']
X = train_set_prepared.drop('Survived', axis=1)

# Clean my Test set

In [66]:
test_set.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [68]:
# Columns to drop
drop_columns = ['PassengerId', 'Cabin', 'Name', 'Ticket']

# Drop column
t_drop_set = test_set.drop(drop_columns, axis=1)

In [71]:
# List categorical variable and numerical varible
cat_set = ['Sex', 'Embarked']
num_set = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

# Set my numerical dataset
t_drop_set_num = t_drop_set[num_set]
t_drop_set_num_attribs = list(t_drop_set_num)
t_drop_set_num_attribs

# Set my categorical variable
t_drop_set_cat =t_drop_set[cat_set]
t_drop_set_cat_attribs = list(drop_set_cat)
#Create a list of cat dataframe
# drop_set_cat_attribs = list(drop_set_cat_trans)
# drop_set_cat_attribs

# Clean my cat Nan rows and convert to dataframe
pipeline = Pipeline([
    ("imputer",SimpleImputer(strategy="most_frequent")),
    ('ord', OrdinalEncoder())
])

t_drop_set_cat_trans = pipeline.fit_transform(t_drop_set_cat)
t_drop_set_cat_trans = pd.DataFrame(t_drop_set_cat_trans, columns=['Sex', 'Embarked'])


# # Deal with NaN in Age column
full_pipeline = ColumnTransformer([
    ('num', SimpleImputer(strategy='median'), t_drop_set_num_attribs),
    ('ord', pipeline, t_drop_set_cat_attribs),
])


# Transform my dataset
test_set_prepared = full_pipeline.fit_transform(t_drop_set)
# Create a column name for my prepared datset
test_set_prepared = pd.DataFrame(test_set_prepared, columns= ['Pclass', 'Age', 'SibSp','Parch', 'Fare', 'Sex','Embarked'])

test_set_prepared.head()

,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
0,3.0,34.5,0.0,0.0,7.8292,1.0,1.0
1,3.0,47.0,1.0,0.0,7.0000,0.0,2.0
2,2.0,62.0,0.0,0.0,9.6875,1.0,1.0
3,3.0,27.0,0.0,0.0,8.6625,1.0,2.0
4,3.0,22.0,1.0,1.0,12.2875,0.0,2.0


In [82]:
#Load the model
svr_linear = SVR(kernel='linear', C=1, gamma='auto')

svr_fit = svr_linear.fit(X, y)

test_set


NameError: name 'accuracy_score' is not defined

In [74]:
y_pred = svr_fit.predict(test_set_prepared)

# Try using RandomizedSearchCV

In [132]:
#Set my paramter grid
param_grid = [
    {'C': np.linspace(0.01, 30, 50)},
    {'epsilon': [0.01, 0.1, 1]},
    {'gamma': ['scale', 'auto']},
    {'kernel': ['rbf','linear',"poly","sigmoid"]}
]

# Load my model
svr_rbf = SVR()

# Finding the best parameters
grid_search = RandomizedSearchCV(svr_rbf, param_grid, cv=5, 
                                 scoring='neg_mean_squared_error', 
                                 return_train_score=True)
# Fit my model
grid_search.fit(X, y)

# Find the best estimator and parameter
best_estimator = grid_search.best_estimator_
best_param = grid_search.best_params_

#Evaluation scores
cvres = grid_search.cv_results_

for mean_scores, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_scores),params)

0.416427640638861 {'C': 25.715714285714288}
13.786637650434162 {'kernel': 'sigmoid'}
0.43270654661080693 {'C': 25.103673469387758}
0.4157601403708816 {'C': 28.77591836734694}
0.4681967039179341 {'C': 0.01}
0.4249860153120484 {'C': 15.923061224489796}
0.42071176663989635 {'C': 23.879591836734697}
0.41875758654868583 {'C': 26.32775510204082}
0.4178031022071908 {'C': 28.16387755102041}
0.4472214686441989 {'gamma': 'auto'}


My best paramter will be 0.4157601403708816 {'C': 28.77591836734694} because it a low rmse in it test score

In [172]:
# find the feature importance
feature_importance = permutation_importance(grid_search,X, y)
feature_importance_value = feature_importance['importances_mean']
feature_importance_value

array([0.01542174, 0.03455791, 0.01507432, 0.00151585, 0.02695813,
       0.12912836, 0.00126251])

In [190]:
#Predicting Surival Rate
y_pred = grid_search.predict(test_set_prepared)

# Converting predicted value to binary
binary_prediction = (y_pred >= 0.5).astype(int) # I got just this code from AI

# Print my accurcacy score
print(classification_report(binary_prediction, label_set['Survived']))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95       250
           1       0.96      0.87      0.91       168

    accuracy                           0.93       418
   macro avg       0.94      0.92      0.93       418
weighted avg       0.93      0.93      0.93       418
